In [1]:
#mean shift
Başlangıç Noktası: İlk olarak, takip etmek istediğiniz nesnenin bir başlangıç pozisyonunu (örneğin bir dikdörtgen) belirlersiniz.
    
    
Yoğunluk Hesaplama: Belirli bir bölgedeki piksellerin yoğunluğunu hesaplamak için bir kernel (örneğin bir Gauss dağılımı) kullan
ılır. Bu işlem, nesnenin rengi veya özellikleri hakkında bilgi toplamak için yapılır.

Hareket: MeanShift algoritması, piksel yoğunluğunun en yüksek olduğu yöne doğru hareket eder. Bu, nesnenin yeni konumunu bulmak
için bir "merkez" hesaplamayı içerir.
Tekrar: Bu süreç, yeni merkez bulunana kadar (veya belirli bir durma koşulu sağlanana kadar) tekrarlanır.
    
Avantajları:

Basit ve hızlıdır.
Belirli bir renk veya özellikteki nesneleri takip etmekte etkilidir.
Dezavantajları:

Çok hızlı hareket eden nesnelerde başarısız olabilir.
Çeşitli nesnelerin rengi benzer olduğunda karışıklığa neden olabilir.



#cam shift 
Başlangıç Noktası: İlk olarak, takip etmek istediğiniz nesnenin başlangıç konumunu (bir dikdörtgen) belirleyin.
    
Yoğunluk Hesaplama: Aynı şekilde, nesnenin piksel yoğunluğunu belirlemek için bir histogram oluşturulur.
    
Adaptif Güncelleme: CamShift, her bir döngüde nesnenin boyutunu ve yönünü güncelleyebilir. Bu, nesnenin konumunu bulmanın yanı
sıra, nesnenin boyutundaki değişiklikleri de hesaba katar.

Merkez ve Boyut Belirleme: Algoritma, yoğunluk merkezini belirledikten sonra, nesnenin boyutunu da güncelleyerek daha doğru 
bir takip sağlar.

Avantajları:

Nesnenin boyutundaki değişiklikleri izleyebilir (büyüme veya küçülme).
Daha iyi doğruluk ve kararlılık sağlar.
Dezavantajları:

Daha karmaşık ve işlemci açısından daha yoğun olabilir.
Renk ve doku gibi özelliklerin sürekli güncellenmesi gerekir.
Özet
MeanShift: Basit, hızlı ve yoğunluk temelli bir nesne takip algoritmasıdır. Ancak, nesne boyutundaki değişiklikleri
göz ardı eder.

CamShift: MeanShift'in geliştirilmiş bir versiyonudur ve nesnenin boyutunu ve yönünü sürekli olarak güncelleyerek 
daha esnek bir takip sağlar.


#ağırlıklı ortalama ile yakındaki noktaların çoğunu nerede olduğunu kümeleme ile belirleyip yönünü hesaplayacaz 
##sslere bak


SyntaxError: unterminated string literal (detected at line 46) (4008107202.py, line 46)

In [1]:
import cv2 
import numpy as np

# Capture a video stream
cap = cv2.VideoCapture(0)

# take first frame of the video
ret, frame = cap.read()

# Yüz algılama için Haarcascade dosyasını yükle
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Yüzleri algıla
face_rects = face_cascade.detectMultiScale(frame)

# Eğer yüz algılanmazsa uyarı ver
if len(face_rects) == 0:
    print("Yüz algılanamadı!")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Convert this list of a single array to a tuple of (x, y, w, h)
(face_x, face_y, w, h) = tuple(face_rects[0]) 
track_window = (face_x, face_y, w, h)

# Set up the ROI for tracking
roi = frame[face_y:face_y+h, face_x:face_x+w]

# Use the HSV Color Mapping
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Find histogram to backproject the target on each frame for calculation of meanshift
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0, 180])

# Normalize the histogram array values given a min of 0 and max of 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by at least 1 pt
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if ret == True:
        # Grab the Frame in HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Calculate the Back Projection based off the roi_hist created earlier
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        # Apply meanshift to get the new coordinates of the rectangle
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw the new rectangle on the image
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 5)

        cv2.imshow('img2', img2)

        k = cv2.waitKey(100) & 0xff  Buradaki değeri artırmak yenileme hızını yavaşlatabilir.
        if k == 27:
            break

    else:
        break
# Önceki ve şimdiki yüz koordinatlarını karşılaştır


cv2.destroyAllWindows()
cap.release()


NameError: name 'prev_face_x' is not defined

In [ ]:
u satır, yüzleri algılar. detectMultiScale fonksiyonu, frame içindeki yüzleri arar ve her bir yüzün pozisyonunu,
(x, y, w, h) değerleriyle döndürür.

(x, y) yüzün üst sol köşesini temsil eder.
w ve h ise yüzün genişliği ve yüksekliğidir.


Yüzün Koordinatları: Bu satırda, face_rects tarafından döndürülen ilk yüzün (x, y, w, h) koordinatları alınıyor.
Bu koordinatlar, yüzün bulunduğu dikdörtgeni tanımlar.

(x, y): Yüzün üst sol köşesinin koordinatlarıdır.
w: Yüzün genişliği.
h: Yüzün yüksekliği.
    
Takip Penceresi: track_window değişkeni, bu (x, y, w, h) koordinatlarını bir tuple haline getirir ve yüzü takip etmek için 
başlangıçta kullanılacak bölgeyi tanımlar. Bu pencere, yüzün başlangıçtaki konumudur.

Örnek: Eğer face_rects[0] şu değerlere sahipse:

python
Kodu kopyala
face_rects[0] = [150, 100, 50, 70]
Bu durumda:

face_x = 150, face_y = 100 olur.
w = 50, h = 70 olur.
track_window = (150, 100, 50, 70) olur, yani yüz 150x100 konumunda ve 50 genişliğinde, 70 yüksekliğinde bir 
dikdörtgenle işaretlenir.



 Eğer yüzün track_window değeri (150, 100, 50, 70) ise, bu satır:

face_y = 100 ve h = 70 olduğu için satır aralığı 100:100+70 olur.
face_x = 150 ve w = 50 olduğu için sütun aralığı 150:150+50 olur. Bu durumda, yüzün bulunduğu bölge frame[100:170, 150:200] olarak belirlenir
    . Bu bölgeyi kullanarak yüzün renk özellikleri incelenecek.
    
    
    
HSV Renk Uzayına Dönüştürme: cv2.cvtColor fonksiyonu, ROI'yi (yüzün bulunduğu bölgeyi) BGR renk uzayından HSV renk uzayına
dönüştürür. HSV (Hue, Saturation, Value) renk modeli, renklerin ton, doygunluk ve parlaklık bileşenlerini temsil eder. 
Bu model, renk tabanlı takip algoritmaları için daha uygundur çünkü parlaklık değişikliklerinden daha az etkilenir.


Hue (Ton): Rengin türü (0-179 arası değer alır).
Saturation (Doygunluk): Rengin yoğunluğu.
Value (Parlaklık): Rengin ışık seviyesi.
Histogram Hesaplama: cv2.calcHist fonksiyonu, ROI'deki ton (hue) kanalının histogramını çıkarır. Histogram, 
renk dağılımlarını gösterir ve her bir renk tonunun ne kadar sık tekrarlandığını ifade eder.



cv2.calcHist([hsv_roi], [0], None, [180], [0, 180]): Burada 0 kanalı yani Hue (Ton) kanalı alınır ve 0-180 arasında bir
histogram çıkarılır.
Histogramı Normalizasyonu: cv2.normalize fonksiyonu, histogramı 0 ile 255 arasında normalize eder. Bu işlem
, farklı görüntü aydınlatma koşullarına rağmen daha tutarlı bir takip sağlamak için kullanılır.

Örnek:

Bir yüz bölgesinde, çoğunlukla ten rengi ve biraz saç rengi varsa, histogram şu şekilde olabilir:
Hue (Ton): 0-50 arası değerlerde bir yoğunluk olabilir (ten rengi tonları).
Saturation (Doygunluk): Yüzdeki farklı renk yoğunluklarını gösterir.
Value (Parlaklık): Parlak bölgeleri gösterir (gözler veya yüz parlak alanları).
    
    
    
    
Her Karede Yüz Takibi: Bu döngüde, her yeni karede yüzü takip etmek için işlemler yapılır.

HSV'ye Dönüştürme: Her kare (frame), yüzün renk özelliklerini daha kolay izleyebilmek için HSV renk uzayına dönüştürülür.

Back Projection (Geri Projeksiyon): cv2.calcBackProject fonksiyonu, önceki adımda elde edilen ROI histogramını kullanarak,
her karede yüzün olabileceği bölgeleri tahmin eder. Bu tahminler, görüntüdeki her bir pikselin histogramla ne kadar eşleştiğini 
gösterir.



Geri Projeksiyon Nedir? Histogram ile her pikselin uyumluluğunu hesaplayarak, hedef nesnenin (yüz) olabileceği bölgeleri geri
projekte eder. Yani, histogram bilgisi kullanılarak yüzün yeni konumu belirlenir.


MeanShift Algoritması: cv2.meanShift fonksiyonu, yüzün yeni konumunu bulmak için MeanShift algoritmasını kullanır.
    MeanShift, geri projeksiyon sonuçlarına göre yüzün hareket ettiği yeni bölgeyi bulur ve takip penceresini (track_window) 
günceller. Yani, yüz hareket ettikçe yeni (x, y, w, h) koordinatları bulunur.


Örnek:

Örneğin, yüz başlangıçta (150, 100, 50, 70) konumundaysa ve geri projeksiyon sonucu yüzün yeni konumu (160, 110, 50, 70) olarak hesaplandıysa, track_window bu yeni konuma taşınacaktır.
MeanShift Algoritması Hakkında:

MeanShift algoritması, bir hedef nesnenin (bu durumda yüz) hareketini takip etmek için yoğunluk bazlı bir algoritmadır. Piksel değerlerinin yoğunluğunu dikkate alarak yüzün olabileceği en yoğun bölgeye hareket eder. Bu sayede, yüzün hangi yöne hareket ettiğini belirlemek mümkündür.
